In [1]:
#导包
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import time	#用来计算训练用时

In [3]:
#导入数据
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
trainset = torchvision.datasets.CIFAR10('./data',train=True, download=True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0)
testset = torchvision.datasets.CIFAR10('./data',train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, num_workers=0)

#查看trainloader中一个batch
features, targets = next(iter(trainloader)) #从dataloader中取出一个batch
print(features.shape)
print(targets.shape)
#用作训练中的验证
testdata_iter = iter(testloader)
test_image, test_label = next(testdata_iter)

#定义模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Files already downloaded and verified
Files already downloaded and verified
torch.Size([16, 3, 32, 32])
torch.Size([16])


In [4]:
model=LeNet()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
#开始训练
start_time = time.time()
for epoch in range(5):
    running_loss = 0.0
    for step, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        
        if step%500 == 499:
            with torch.no_grad():
                test_outputs = model(test_image)
                predict = torch.max(test_outputs,dim=1)[1]
                accuracy = torch.eq(predict,test_label).sum().item()/test_label.size(0)
                print("[{},{}] train_loss:{} test_acc:{}".format(epoch+1,step+1,running_loss/500,accuracy))
                running_loss=0.0
                

[1,500] train_loss:1.9355148510932922 test_acc:0.4375
[1,1000] train_loss:1.6645738188028336 test_acc:0.4375
[1,1500] train_loss:1.572817909359932 test_acc:0.4375
[1,2000] train_loss:1.4906463536024093 test_acc:0.5625
[1,2500] train_loss:1.4499745008945466 test_acc:0.5
[1,3000] train_loss:1.4021641203165054 test_acc:0.5625
[2,500] train_loss:1.3254381021261215 test_acc:0.5
[2,1000] train_loss:1.3072820085287093 test_acc:0.625
[2,1500] train_loss:1.3004584715366363 test_acc:0.5625
[2,2000] train_loss:1.2861582061052323 test_acc:0.75
[2,2500] train_loss:1.265197258591652 test_acc:0.625
[2,3000] train_loss:1.2586927539110184 test_acc:0.625
[3,500] train_loss:1.1916196236014367 test_acc:0.6875
[3,1000] train_loss:1.1612997405529022 test_acc:0.5625
[3,1500] train_loss:1.1612069365978241 test_acc:0.8125
[3,2000] train_loss:1.1778837076425552 test_acc:0.75
[3,2500] train_loss:1.1524815465211868 test_acc:0.75
[3,3000] train_loss:1.169780957400799 test_acc:0.8125
[4,500] train_loss:1.0939181171

In [6]:
end_time = time.time()
total_time = end_time - start_time
print('total_time:{}'.format(total_time))

total_time:129.2853808403015
